In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function

import math

import numpy as np
import matplotlib.pyplot as plt
from pytest import approx

from fastimgproto.fixtures.image import (
    add_gaussian2d_to_image,
    gaussian_point_source,
)
from fastimgproto.fixtures.sourcefind import random_sources_on_grid
from fastimgproto.sourcefind.fit import Gaussian2dParams
from fastimgproto.sourcefind.image import SourceFindImage

from fastimgproto.fixtures.sourcefind import (
    generate_random_source_params,
    check_single_source_extraction_successful,
)

import logging
import datetime

logger = logging.getLogger(__name__)

In [ ]:
amplitude_range = (6., 42.)
semiminor_range = (1.2, 2.5)
axis_ratio_range = (1., 2.)
seed = 123456

In [ ]:
image_size = 1024
n_sources = 64

In [ ]:
def generate_test_image(image_size, n_sources, seed=None):
    image = np.zeros((image_size, image_size), dtype=np.float_)
    sources = random_sources_on_grid(image_size,
                                     n_sources,
                                     amplitude_range=amplitude_range,
                                     semiminor_range=semiminor_range,
                                     axis_ratio_range=axis_ratio_range,
                                     seed=seed
                                     )
    for s in sources:
        add_gaussian2d_to_image(s, image)
    return image

In [ ]:
image = generate_test_image(image_size=2048, n_sources=64, seed=seed)

In [ ]:
def time_native_sourcefinding(image):
    start = datetime.datetime.now()
    sfimg = SourceFindImage(image, detection_n_sigma=5, analysis_n_sigma=3, rms_est=1.)
    extraction_complete = datetime.datetime.now()
    sfimg.fit_islands()
    fitting_complete = datetime.datetime.now()
    extraction_duration =  (extraction_complete - start).total_seconds()
    total_duration =  (fitting_complete - start).total_seconds()
    return extraction_duration, total_duration

In [ ]:
time_native_sourcefinding(image)

In [ ]:
%%time
images_w_64_sources = [ generate_test_image(image_size=sz, n_sources=64)
                      for sz in (512, 1024, 2048, 4096)]